In [2]:
import pysam
import Bio
from Bio import SeqIO
from Bio.SeqUtils import GC
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import pandas as pd

In [3]:
def samfile_stats(samfile):
    samfile = pysam.AlignmentFile(samfile, "r")
    iter = samfile.fetch()
    total = 0
    non_neg_pos = 0
    ref_regions = []
    no_hits = []
    pos = []
    for x in iter:
        total += 1
        if x.reference_start > -1:
            ref_regions.append(x.reference_id)
            pos.append(x.reference_start)
            non_neg_pos += 1
        else:
            no_hits.append(x.reference_id)
    
    print("ALIGNMENT RATE: ", non_neg_pos / total)
    print("REGIONS: ", set(ref_regions))
    print("NO HIT REGIONS: ", set(no_hits))
    
    return pd.DataFrame({"REGIONS":  ref_regions, "POSITION": pos})

In [20]:
# Alignment results from seed length of 19
samfile_stats("align_out.sam")

ALIGNMENT RATE:  0.8010200876820267
REGIONS:  {0, 1, 2}
NO HIT REGIONS:  {-1}


In [32]:
# Alignment results from seed length of 15 (BEST)
df = samfile_stats("align_seed_15.sam")

ALIGNMENT RATE:  0.8129087183881273
REGIONS:  {0, 1, 2}
NO HIT REGIONS:  {-1}


In [18]:
# Alignment results from seed length of 10
samfile_stats("align_seed_10.sam")

ALIGNMENT RATE:  0.815672533218076
REGIONS:  {0, 1, 2}
NO HIT REGIONS:  {-1}


In [34]:
# dataframe df contains the regions and read hits for seed length of 15 (best seed)
df.head()

,REGIONS,POSITION
0,0,904151
1,0,904065
2,0,1257417
3,0,1256912
4,0,827685


In [52]:
read_hits = []
with open("outfile_cleaned", "r") as ref_info:
    for line in ref_info:
        info = line.split(" ")
        read_hits.append((info[0], int(info[1])-1))

In [53]:
read_hits

[('NZ_CP009617.1', 904151),
 ('NZ_CP009617.1', 904065),
 ('NZ_CP009617.1', 1257417),
 ('NZ_CP009617.1', 1256912),
 ('NZ_CP009617.1', 827685),
 ('NZ_CP009617.1', 827668),
 ('NZ_CP009617.1', 129644),
 ('NZ_CP009617.1', 129476),
 ('NZ_CP009618.1', 531766),
 ('NZ_CP009618.1', 531367),
 ('NZ_CP009617.1', 3353498),
 ('NZ_CP009617.1', 3353006),
 ('NZ_CP009618.1', 95652),
 ('NZ_CP009618.1', 95204),
 ('NZ_CP009618.1', 1183791),
 ('NZ_CP009618.1', 1183408),
 ('NZ_CP009617.1', 181513),
 ('NZ_CP009617.1', 181364),
 ('NZ_CP009617.1', 2533009),
 ('NZ_CP009617.1', 2532687),
 ('NZ_CP009617.1', 77277),
 ('NZ_CP009617.1', 77293),
 ('NZ_CP009617.1', 3353252),
 ('NZ_CP009617.1', 3353402),
 ('NZ_CP009617.1', 3011191),
 ('NZ_CP009617.1', 3011496),
 ('NZ_CP009618.1', 1171375),
 ('NZ_CP009618.1', 1171320),
 ('NZ_CP009617.1', 819097),
 ('NZ_CP009617.1', 818753),
 ('NZ_CP009617.1', 964004),
 ('NZ_CP009617.1', 963846),
 ('NZ_CP009617.1', 1980293),
 ('NZ_CP009617.1', 1979929),
 ('NZ_CP009618.1', 1417664),
 ('NZ_C

In [86]:
L = 100
ref = []
windows = []
gc = []
chunks = []
with open("ref1.fna", "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        chunks += [(record.id, record.seq[i:i+L], (i,i+L-1)) for i in range(0, len(record.seq), L)]

In [87]:
gc = [[chunks[i][0], chunks[i][2], GC(chunks[i][1])] for i in range(len(chunks))]

In [88]:
gc

[['NZ_CP009617.1', (0, 99), 39.0],
 ['NZ_CP009617.1', (100, 199), 35.0],
 ['NZ_CP009617.1', (200, 299), 30.0],
 ['NZ_CP009617.1', (300, 399), 36.0],
 ['NZ_CP009617.1', (400, 499), 43.0],
 ['NZ_CP009617.1', (500, 599), 51.0],
 ['NZ_CP009617.1', (600, 699), 48.0],
 ['NZ_CP009617.1', (700, 799), 47.0],
 ['NZ_CP009617.1', (800, 899), 47.0],
 ['NZ_CP009617.1', (900, 999), 53.0],
 ['NZ_CP009617.1', (1000, 1099), 47.0],
 ['NZ_CP009617.1', (1100, 1199), 48.0],
 ['NZ_CP009617.1', (1200, 1299), 50.0],
 ['NZ_CP009617.1', (1300, 1399), 48.0],
 ['NZ_CP009617.1', (1400, 1499), 41.0],
 ['NZ_CP009617.1', (1500, 1599), 42.0],
 ['NZ_CP009617.1', (1600, 1699), 49.0],
 ['NZ_CP009617.1', (1700, 1799), 55.0],
 ['NZ_CP009617.1', (1800, 1899), 49.0],
 ['NZ_CP009617.1', (1900, 1999), 47.0],
 ['NZ_CP009617.1', (2000, 2099), 53.0],
 ['NZ_CP009617.1', (2100, 2199), 46.0],
 ['NZ_CP009617.1', (2200, 2299), 48.0],
 ['NZ_CP009617.1', (2300, 2399), 46.0],
 ['NZ_CP009617.1', (2400, 2499), 42.0],
 ['NZ_CP009617.1', (250

In [80]:
def get_hit_counts(sam_hits):
    hit_count_dict = {}
    for (hid, pos) in sam_hits:
        if not (hid, pos) in hit_count_dict.keys():
            hit_count_dict[(hid, pos)] = 1
        else:
            hit_count_dict[(hid, pos)] += 1
    return hit_count_dict

In [81]:
def get_full_results(sliding_windows, hit_count_dict):
    results = []
    for window in sliding_windows:
        count = 0
        for i in range(window[1][0], window[1][1]+1):
            try:
                count += hit_count_dict[(window[0], i)]
            except Exception:
                None
        results.append(window + [count])
    return results

In [89]:
hits = get_hit_counts(read_hits)

In [90]:
results = get_full_results(gc, hits)

In [91]:
results

[['NZ_CP009617.1', (0, 99), 39.0, 252],
 ['NZ_CP009617.1', (100, 199), 35.0, 122],
 ['NZ_CP009617.1', (200, 299), 30.0, 125],
 ['NZ_CP009617.1', (300, 399), 36.0, 98],
 ['NZ_CP009617.1', (400, 499), 43.0, 89],
 ['NZ_CP009617.1', (500, 599), 51.0, 94],
 ['NZ_CP009617.1', (600, 699), 48.0, 94],
 ['NZ_CP009617.1', (700, 799), 47.0, 77],
 ['NZ_CP009617.1', (800, 899), 47.0, 59],
 ['NZ_CP009617.1', (900, 999), 53.0, 67],
 ['NZ_CP009617.1', (1000, 1099), 47.0, 56],
 ['NZ_CP009617.1', (1100, 1199), 48.0, 78],
 ['NZ_CP009617.1', (1200, 1299), 50.0, 77],
 ['NZ_CP009617.1', (1300, 1399), 48.0, 71],
 ['NZ_CP009617.1', (1400, 1499), 41.0, 66],
 ['NZ_CP009617.1', (1500, 1599), 42.0, 61],
 ['NZ_CP009617.1', (1600, 1699), 49.0, 70],
 ['NZ_CP009617.1', (1700, 1799), 55.0, 70],
 ['NZ_CP009617.1', (1800, 1899), 49.0, 62],
 ['NZ_CP009617.1', (1900, 1999), 47.0, 76],
 ['NZ_CP009617.1', (2000, 2099), 53.0, 75],
 ['NZ_CP009617.1', (2100, 2199), 46.0, 91],
 ['NZ_CP009617.1', (2200, 2299), 48.0, 75],
 ['NZ_CP

In [92]:
data = pd.DataFrame(results, columns=['REFID', 'window', 'GC', 'hits'])

In [98]:
low_gc = data[data.GC <= 60]
high_gc = data[data.GC > 60]

In [99]:
low_gc.hits.mean()

63.90254089222424

In [100]:
high_gc.hits.mean()

13.643109540636042

In [102]:
low_gc.hits.std()

42.86246995027107

In [103]:
high_gc.hits.std()

28.573592558775328